In [2]:
import nltk
nltk.download('punkt') # For tokenization
nltk.download('averaged_perceptron_tagger') # For POS tagging (optional but useful)
nltk.download('stopwords') # Common words to ignore (optional)

[nltk_data] Downloading package punkt to C:\Users\Karthick
[nltk_data]     J\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Karthick J\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to C:\Users\Karthick
[nltk_data]     J\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# chatbot.py
import data_fetcher # Import our data functions
import nltk
import random

# Basic keywords for intents
GREETING_KEYWORDS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
FAREWELL_KEYWORDS = ("bye", "goodbye", "quit", "exit", "see ya")
DEFINITION_KEYWORDS = ("define", "definition", "what is", "explain")
DATA_KEYWORDS = ("latest", "value", "number", "figure", "data", "rate", "level")

# Simple responses
GREETING_RESPONSES = ["Hello!", "Hi there!", "Greetings!", "How can I help you with macroeconomic data today?"]
FAREWELL_RESPONSES = ["Goodbye!", "See you later!", "Have a great day!"]
UNKNOWN_RESPONSES = [
    "Sorry, I didn't quite understand that. Can you rephrase?",
    "I'm not sure how to respond to that. Try asking for the 'latest value' or 'definition' of an indicator like GDP, CPI, or Unemployment Rate.",
    "My apologies, I can only provide definitions or latest values for specific indicators right now."
    ]

def process_input(user_input):
    """
    Analyzes user input to determine intent and entities (basic version).
    Returns the chatbot's response.
    """
    user_input_lower = user_input.lower()
    tokens = nltk.word_tokenize(user_input_lower) # Split into words

    # 1. Check for Greetings
    if any(word in tokens for word in GREETING_KEYWORDS):
        return random.choice(GREETING_RESPONSES)

    # 2. Check for Farewells
    if any(word in tokens for word in FAREWELL_KEYWORDS):
        return random.choice(FAREWELL_RESPONSES) # Signal to exit loop later

    # 3. Identify Intent (Definition vs. Data) & Entity (Indicator)
    intent = None
    entity = None

    if any(word in tokens for word in DEFINITION_KEYWORDS):
        intent = "get_definition"
    elif any(word in tokens for word in DATA_KEYWORDS):
        intent = "get_data"

    # Find the indicator (entity) - check against our known indicators
    for indicator_keyword in data_fetcher.indicator_map.keys():
        # Check if the full keyword phrase is in the input
        if indicator_keyword in user_input_lower:
            entity = indicator_keyword
            break # Take the first match found

    # 4. Generate Response based on Intent and Entity
    if intent == "get_definition" and entity:
        return data_fetcher.get_definition(entity)
    elif intent == "get_data" and entity:
        indicator_details = data_fetcher.indicator_map.get(entity)
        if indicator_details:
            if indicator_details["source"] == "fred":
                return f"The latest {entity} is: {data_fetcher.get_fred_series_latest(indicator_details['id'])}"
            elif indicator_details["source"] == "worldbank":
                # Add country handling if needed later
                country = indicator_details.get('country', 'US')
                return f"The latest {entity} (World Bank, {country}) is: {data_fetcher.get_world_bank_indicator_latest(indicator_details['id'], country)}"
            else:
                return f"Sorry, I don't know how to fetch data for {entity} from '{indicator_details['source']}' yet."
        else:
            # This case shouldn't happen if entity mapping is correct, but good to have
            return f"Sorry, I recognized '{entity}' but couldn't find how to fetch its data."

    # Fallback if intent/entity not clear or combination is invalid
    if entity and not intent: # User mentioned indicator but not what they want
         return f"What would you like to know about {entity}? Try asking for the 'latest value' or 'definition'."
    elif intent and not entity: # User asked for data/definition but not which indicator
        return "Which indicator are you interested in (e.g., Real GDP, CPI, Unemployment Rate)?"
    else: # Couldn't understand
        return random.choice(UNKNOWN_RESPONSES)

# Main chat loop
if __name__ == "__main__":
    print(random.choice(GREETING_RESPONSES) + " (Type 'bye' to exit)")
    while True:
        user_text = input("You: ")
        response = process_input(user_text)
        print(f"Bot: {response}")

        # Check if the response is a farewell to break the loop
        if response in FAREWELL_RESPONSES:
            break